In [1]:
import datetime
from pyspark.sql import DataFrame, Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, col
from pyspark.sql import functions as F
import sys
from hops import hdfs as hdfs
print(hdfs.project_path())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
23,application_1613204927129_0032,pyspark,idle,Link,Link


SparkSession available as 'spark'.
hdfs://rpc.namenode.service.consul:8020/Projects/pittre/

In [2]:
fg1_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f1", StringType(), True)    
])

fg1=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/1000000-2000-2-out.csv", header=True, schema=fg1_schema)
fg1=fg1.sort(col("id"),col("ts"))
fg1.show()

+---+----+-----+
| id|  ts|   f1|
+---+----+-----+
|  1|1002| f2-1|
|  1|1004| f2-2|
|  1|1006| f2-3|
|  1|1008| f2-4|
|  1|1010| f2-5|
|  1|1012| f2-6|
|  1|1014| f2-7|
|  1|1016| f2-8|
|  1|1018| f2-9|
|  1|1020|f2-10|
|  1|1022|f2-11|
|  1|1024|f2-12|
|  1|1026|f2-13|
|  1|1028|f2-14|
|  1|1030|f2-15|
|  1|1032|f2-16|
|  1|1034|f2-17|
|  1|1036|f2-18|
|  1|1038|f2-19|
|  1|1040|f2-20|
+---+----+-----+
only showing top 20 rows

In [3]:
fg2_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f2", StringType(), True)    
])

fg2=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/1000000-2000-3-out.csv", header=True, schema=fg2_schema)
fg2=fg2.select([col("id").alias("id_2"), col("ts").alias("ts_2"), col("f2")])
fg2.show()

+----+----+-----+
|id_2|ts_2|   f2|
+----+----+-----+
|   1|1003| f3-1|
|   1|1006| f3-2|
|   1|1009| f3-3|
|   1|1012| f3-4|
|   1|1015| f3-5|
|   1|1018| f3-6|
|   1|1021| f3-7|
|   1|1024| f3-8|
|   1|1027| f3-9|
|   1|1030|f3-10|
|   1|1033|f3-11|
|   1|1036|f3-12|
|   1|1039|f3-13|
|   1|1042|f3-14|
|   1|1045|f3-15|
|   1|1048|f3-16|
|   1|1051|f3-17|
|   1|1054|f3-18|
|   1|1057|f3-19|
|   1|1060|f3-20|
+----+----+-----+
only showing top 20 rows

In [4]:
from pyspark.sql import Window
from pyspark.sql.functions import lit, when, col, lag, rank

win = Window.partitionBy(["id", "ts"]).orderBy(col('ts_2').desc())

In [5]:
final=fg1.join(fg2, (fg1.id == fg2.id_2) & (fg1.ts >= fg2.ts_2)) \
    .withColumn("id_rank", rank().over(win)) \
    .filter(col("id_rank") == 1).drop(col("id_rank")).drop(col("ts_2")).drop(col("id_2"))

In [6]:
final.show()

+---+----+---------+--------+
| id|  ts|       f1|      f2|
+---+----+---------+--------+
|148|1004|f2-147002|f3-97903|
|148|1006|f2-147003|f3-97904|
|148|1008|f2-147004|f3-97904|
|148|1010|f2-147005|f3-97905|
|148|1012|f2-147006|f3-97906|
|148|1014|f2-147007|f3-97906|
|148|1016|f2-147008|f3-97907|
|148|1018|f2-147009|f3-97908|
|148|1020|f2-147010|f3-97908|
|148|1022|f2-147011|f3-97909|
|148|1024|f2-147012|f3-97910|
|148|1026|f2-147013|f3-97910|
|148|1028|f2-147014|f3-97911|
|148|1030|f2-147015|f3-97912|
|148|1032|f2-147016|f3-97912|
|148|1034|f2-147017|f3-97913|
|148|1036|f2-147018|f3-97914|
|148|1038|f2-147019|f3-97914|
|148|1040|f2-147020|f3-97915|
|148|1042|f2-147021|f3-97916|
+---+----+---------+--------+
only showing top 20 rows

In [7]:
final.count()

207920871